#Introduction
This tutorial covers the following topics:
*   Loading 3D images (NIFTI) as input to a CNN
*   Developing an encoder-decorder segmentation network (U-Net)

The data used in this exercise includes CT images and lung contours downloaded from The Cancer Imaging Archive: 
> _Yang, Jinzhong; Sharp, Greg; Veeraraghavan, Harini ; van Elmpt, Wouter ; Dekker, Andre; Lustberg, Tim; Gooding, Mark. (2017). Data from Lung CT Segmentation Challenge. The Cancer Imaging Archive. http://doi.org/10.7937/K9/TCIA.2017.3r3fvz08_

Three-dimensionsal CT images and lung contours/masks were converted into the NIFTI file format. CT images were normalized such (-1000 HU, 1000 HU) became (0, 1).  

Note: make sure your session is connected to a GPU. From the dropdown menu: __Runtime > Change Runtime Type > GPU (or TPU)__

#Data loading
First we need to download the data (hosted in cloud storage) into our Colab session. <br>
Then we untar and decompress it "locally" to our /home/ folder.

In [0]:
# download the data from the cloud
! wget -N https://f001.backblazeb2.com/file/snmmi-hands-on-ai/LCTSC_nii.tar.gz -P /home

# this should take a few seconds, please be patient
print('decompressing...')
! tar xzf "/home/LCTSC_nii.tar.gz" --directory /home
print('done!')

Import some libraries. Nibabel is our library to handle NIFTI files

In [0]:
import nibabel as nib
import numpy as np
import os
import random

Each folder in /home/LCTSC_nii/  contains data for a different subject:
> /home/LCTSC_nii/  
 &nbsp;&nbsp;&nbsp;001/  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ct.nii  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;mask.nii  
 &nbsp;&nbsp;&nbsp;002/  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;ct.nii  
 &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;mask.nii  
 &nbsp;&nbsp;&nbsp;             ...

Store paths to all subjects' data to a list, then randomize the orderering

In [0]:
path_to_images='/home/LCTSC_nii'
subject_paths = [i.path for i in os.scandir(path_to_images) if i.is_dir()]

#randomize order
random.seed(1)
random.shuffle(subject_paths)
print(subject_paths)

#which subjects should be validation 
#(note: careful not to include a subject's images in both train and validation)
validation_subjects = [0, 1, 2]

Loop through each subject and do the following:


*   Read each subject's NIFTI images and masks
*   Resize them 
*   Convert them into format of (samples, x_dim, y_dim, channels), treating each axial slice as a sample
*   Sort them into training and validation data sets

Note: your projects will likely need an additional testing data set


In [0]:
#use CV2 to resample the images
from cv2 import resize, INTER_LINEAR, INTER_NEAREST

resample_dims = 128

#initialize arrays (can erase later, but probably doesn't matter)
train_images = np.zeros((1,resample_dims, resample_dims,1))
train_masks =  np.zeros((1,resample_dims, resample_dims,1))
val_images = np.zeros((1,resample_dims, resample_dims,1))
val_masks = np.zeros((1,resample_dims, resample_dims,1))

#loop through, loading each subject's data
for i,subject in enumerate(subject_paths):

  #read a subject's stack of 2D images and stack of 2D masks, resize
  image_stack_i = resize(nib.load(os.path.join(subject, 'ct.nii')).get_fdata(),(resample_dims,resample_dims), interpolation=INTER_LINEAR) 
  mask_stack_i = resize(nib.load(os.path.join(subject, 'mask.nii')).get_fdata(),(resample_dims,resample_dims), interpolation=INTER_NEAREST)
  
  #rearrange the data from (x_dim, y_dim, samples) into (samples, x_dim, y_dim)  
  image_stack_i = np.moveaxis(image_stack_i, 2,0)
  mask_stack_i = np.moveaxis(mask_stack_i, 2,0)

  #this adds the channels dimension (may seem strange, but it is needed)
  image_stack_i = image_stack_i[:,:,:,np.newaxis]
  mask_stack_i = mask_stack_i[:,:,:,np.newaxis]

  #add images and masks, split into training and validation
  if i in validation_subjects:
    val_images = np.concatenate((val_images, image_stack_i), axis=0) 
    val_masks = np.concatenate((val_masks, mask_stack_i), axis=0)
  else:
    train_images = np.concatenate((train_images, image_stack_i), axis=0) 
    train_masks = np.concatenate((train_masks, mask_stack_i), axis=0)


In [0]:
print('Size of input images: ' + str(train_images.shape))
print('Size of input masks: ' + str(train_masks.shape))
print('Size of validation images: ' + str(val_images.shape))
print('Size of validation masks: ' + str(val_masks.shape))

Let's inspect the images, to make sure they look like what we expect (very important!). You can run this block many times with a random slice being shown each time it's run. 

In [0]:
from matplotlib import pyplot as plt
# set plotting to be in-line
%matplotlib inline

#view random slice, concatenating the image and mask next to each other
random_slice = random.randint(0,train_images.shape[0])
fig = plt.figure(figsize=(10,10))
plt.imshow(np.concatenate((train_images[random_slice,:,:,0], train_masks[random_slice,:,:,0]),axis=1), cmap='gray')
plt.show()

#Model building and training
First load some essential libraries for model building

In [0]:
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import UpSampling2D
from tensorflow.keras.layers import concatenate
# also, import the Model function for building the model
from tensorflow.keras import Model

Build a 5-level encoder module (ie, 4 down-sampling steps) with skip connections to a mirror-image decoder module.
The number of filters can be altered -- this is a "small" version of the U-Net

In [0]:
# create our input layer by giving it an input shape
input_layer = Input(shape=train_images.shape[1:])

#level 1
conv1_en = Conv2D(8, 3, activation = 'relu', padding = 'same')(input_layer)
pool1_en = MaxPooling2D(pool_size=(2, 2))(conv1_en)
#level 2
conv2_en = Conv2D(16, 3, activation = 'relu', padding = 'same')(pool1_en)
pool2_en = MaxPooling2D(pool_size=(2, 2))(conv2_en)
#level 3
conv3_en = Conv2D(32, 3, activation = 'relu', padding = 'same')(pool2_en)
pool3_en = MaxPooling2D(pool_size=(2, 2))(conv3_en)
#level 4
conv4_en = Conv2D(64, 3, activation = 'relu', padding = 'same')(pool3_en)
pool4_en = MaxPooling2D(pool_size=(2, 2))(conv4_en)

#bottom level
conv_bottom = Conv2D(64, 3, activation = 'relu', padding = 'same')(pool4_en)

#level 4 - DECODER
up4_dec = UpSampling2D(size = (2,2))(conv_bottom)
conv4_dec = concatenate([conv4_en,up4_dec])
conv4_dec = Conv2D(64, 3, activation = 'relu', padding = 'same')(conv4_dec)
#level 3 - DECODER
up3_dec = UpSampling2D(size = (2,2))(conv4_dec)
conv3_dec = concatenate([conv3_en,up3_dec])
conv3_dec = Conv2D(32, 3, activation = 'relu', padding = 'same')(conv3_dec)
#level 2 - DECODER
up2_dec = UpSampling2D(size = (2,2))(conv3_dec)
conv2_dec = concatenate([conv2_en,up2_dec])
conv2_dec = Conv2D(16, 3, activation = 'relu', padding = 'same')(conv2_dec)
#level 1 - DECODER
up1_dec = UpSampling2D(size = (2,2))(conv2_dec)
conv1_dec = concatenate([conv1_en,up1_dec])
conv1_dec = Conv2D(8, 3, activation = 'relu', padding = 'same')(conv1_dec)

#output level
conv_out = Conv2D(8, 3, activation = 'relu', padding = 'same')(conv1_dec)
conv_out = Conv2D(1, 3, activation = 'sigmoid', padding='same')(conv_out)

#define inputs/outpus
unet = Model(inputs = input_layer, outputs = conv_out)

Inspect the model

In [0]:
#show model summary
unet.summary()

In [0]:
#you can also create a figure
from tensorflow.keras.utils import plot_model
plot_model(unet, show_shapes=True)

Compile and train the model. All the training results get stored in _history_ variable

In [0]:
from tensorflow.keras import optimizers

#I tried several combinations, RMSprop with lr=0.001 worked for me
unet.compile(loss='binary_crossentropy',optimizer=optimizers.RMSprop(lr=0.001))

In [0]:
history = unet.fit(x=train_images, y=train_masks,
          batch_size=32,
          epochs=10,
          verbose=1,
          shuffle=True,
          validation_data=(val_images, val_masks))

Now inspect the results. First look at the loss plot. Change the _plt.ylim_ as appropriate

In [0]:
# Plot the losses that are stored in the 'hist' variable
plt.figure(figsize=(10,10));
plt.plot(history.epoch,history.history['loss'],'b-s')
plt.plot(history.epoch,history.history['val_loss'],'r-s')
plt.legend(['Training Loss',
            ' Validation Loss'])
plt.xlabel('Epochs')
plt.ylabel('Binary Crossentropy')
plt.ylim([0,0.1])
plt.show()

Show the images with the results. First run the unet  model using our validation images as input. Then show the input image next to the ground truth mask and the predicted mask.

In [0]:
#apply the unet to our validation images
predictions = unet.predict(val_images, verbose=1)

In [0]:
#show random images with the predicted results
random_slice = random.randint(0,val_images.shape[0])
fig = plt.figure(figsize=(10,10))
im_slice = val_images[random_slice,:,:,0]
true_mask_slice = val_masks[random_slice,:,:,0]
pred_mask_slice = predictions[random_slice,:,:,0]
plt.imshow(np.c_[im_slice, true_mask_slice, np.round(pred_mask_slice)], cmap='gray')
plt.show()

In [0]:
#save your model (optional)
unet.save('/home/unet_lung_segmation_model.hdf5')